In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pandas import Series, DataFrame

C:\Users\hashi\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [31]:
# Upload the data 
opp_per100 = pd.read_table('opp_per100.txt', sep=',', header = 0)
opp_pergame = pd.read_table('opp_pergame.txt', sep=',', header = 0)
opp_shooting = pd.read_table('opp_shooting.txt', sep=',', header = 2)
team_misc = pd.read_table('team_misc.txt', sep=',', header = 1)
team_per100 = pd.read_table('team_per100.txt', sep=',', header = 0)
team_pergame = pd.read_table('team_pergame.txt', sep=',', header = 0)
team_shooting = pd.read_table('team_shooting.txt', sep=',', header = 2)

## Four-Factors 

To win basketball games, Dean Oliver identified the following statistics as being essential, he called these **"Four Factors of Basketball Success"**. The number in the parenthesis is the weight Dean Oliver gave each factor

* Shooting (40%) measured using `eFG%`
* Turnovers (25%) measured using `TOV%`
* Rebounding (20%) measured using `ORB%` and `DRB%`
* Freethrows (15%) measured using `FT%`

Let us see how accurate we are in predicting the number of wins each team had in the 2017-18 season using these factors and weights


## Methodology

I intend on using Weighted Least Sqaures regression, assiging the weights that Dean Oliver originally proposed